In [1]:
%%capture
!pip install wandb -qU

In [2]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [6]:
def make_loader(batch_size, train=True):
    full_dataset = datasets.MNIST(root='./data/MNIST', train=train, 
                                    download=True,  transform=transforms.ToTensor())
    
    loader = DataLoader(dataset=full_dataset,
                        batch_size=batch_size, 
                        shuffle=True,
                        pin_memory=True, num_workers=2)

    return loader

In [7]:
class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [8]:
def train(model, loader, criterion, optimizer, device, config):
    cumu_loss = 0
    tqdm_loader = tqdm(loader)

    for images, labels in tqdm_loader:
        images, labels = images.to(device), labels.to(device)
        logit = model(images)
        loss = criterion(logit, labels)
        cumu_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return loss

In [9]:
def valid(model,test_loader):
    model.eval()
    acc_list = []
    
    with torch.no_grad():
        tqdm_loader = tqdm(test_loader)
        for images, labels in tqdm_loader:
            logit = model(images)
            pred = torch.argmax(logit, dim=1)
            batch_acc = accuracy_score(labels, pred)
            acc_list.append(batch_acc)

    return sum(acc_list) / len(acc_list) 

In [10]:
config  = {
    'epochs': 5,
    'classes':10,
    'batch_size': 4,
    'kernels': [16, 32],
    'weight_decay': 0.0005,
    'learning_rate': 1e-3,
    'dataset': 'MNIST',
    'architecture': 'CNN',
    'val_evrey' : 5,
    'seed': 42
    }

def run(config=None):
    wandb.init(project='MNIST-project', config=config)
    config = wandb.config
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_loader = make_loader(batch_size=config.batch_size, train=True)
    test_loader = make_loader(batch_size=config.batch_size, train=False)

    model = ConvNet(config.kernels, config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(1, config.epochs):
        loss = train(model, train_loader, criterion, optimizer, device, config)
        acc = valid(model, test_loader)
        
        wandb.log({"Loss": loss})
        wandb.log({"Acc": acc})

        print(f"[Train] Epoch {epoch:02} | Loss: {loss:.2f}")
        print(f"[Vaild] Epoch {epoch:02} | Acc: {acc:.3f}")
        

        # if epoch % config.val_evrey  == 0:
        #     acc = valid(model, test_loader)
        #     print(f"[Vaild] Epoch {epoch:02} | Acc: {acc:.3f}")
        #     wandb.log({"Acc": acc})

    return model

In [11]:
# relogin error 발생 시 실행
# !wandb login --relogin

In [12]:
model = run(config)

wandb: Currently logged in as: kcyoon689 (mnist-tutorials). Use `wandb login --relogin` to force relogin


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



...

[Train] Epoch 02 | Loss: 0.10
[Vaild] Epoch 02 | Acc: 0.986


100%|██████████| 79/79 [00:03<00:00, 22.57it/s]


[Train] Epoch 03 | Loss: 0.05
[Vaild] Epoch 03 | Acc: 0.987


100%|██████████| 79/79 [00:04<00:00, 16.55it/s]

[Train] Epoch 04 | Loss: 0.03
[Vaild] Epoch 04 | Acc: 0.989
